In [1]:
from datetime import datetime
import pickle
import PIL, torch, torchvision
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from torchvision import models
from torch import nn
import pandas as pd

from load_data import load_et_maps, load_images, load_labels, classes
from load_models import load_mobilenet, load_resnet
from utils import batch_accuracy, get_targets

if torch.cuda.is_available() == True:
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)
dtype = torch.float32

OCC_ONLY = Path('/home/florian/dev/ET-Adversarials/data/Occ_only_226px/')
IMGS = Path('/home/florian/dev/ET-Adversarials/data/scaled_images/')

#torch.cuda.set_per_process_memory_fraction(.5,0)

cuda:0


# Load Data and Model

In [2]:
img_batch = load_images(IMGS).to(device)
etm_batch = load_et_maps(OCC_ONLY).to(device)
labels = load_labels(IMGS).to(device)


In [3]:
MOBILENET_PATH = './models/MobileNetV3Small.pt'
model = load_mobilenet(MOBILENET_PATH).to(device)
batch_accuracy(model, img_batch, labels)

1.0

In [4]:
targets = get_targets(model, img_batch.to(device), labels).to(device)

# Parallel Implementation

In [5]:
import gc
import eagerpy as ep

def classification_loss(logits,t):
    
    second_most_probable = logits[torch.arange(len(logits)),logits.argsort(descending=True)[:,1]]
    most_probable_class = logits[torch.arange(len(logits)),logits.argmax(-1)]

    t = t.to(device).long()
    loss = torch.ones_like(t)
    #print(logits.shape, second_most_probable.shape)
    target_successful_loss = torch.nn.functional.relu(second_most_probable - logits[torch.arange(len(logits)),t]) 
    target_unsuccessful_loss = torch.nn.functional.relu(most_probable_class - logits[torch.arange(len(logits)),t])
    #print(target_successful_loss.shape,target_unsuccessful_loss.shape, (logits.argmax(-1) == t).shape)
    loss = target_successful_loss.where(logits.argmax(-1) == t, target_unsuccessful_loss)
    
        
    return loss

def hpa_loss(w,x,et_map):
    '''
        This is the loss term added to the original C&W term. It is the product of delta_i and 
        the eye-tracking-value of pixel i. As such, the loss is high for high delta and eye-tracking values, 
        but low otherwise. Effectively incentivizing the algorithm to find perturbations outside the area 
        humans spend most time when evaluating the image.
    '''

    delta = 0.5 * (torch.tanh(w) + 1) - x
    return (delta * et_map).flatten(-3).norm(dim=-1)

def et_loss_fn(w,x, logits, et_map, target):
    l2_loss = ((.5*(torch.tanh(w) + 1)) - x ).flatten(-3).abs().norm(dim=-1)
    class_loss = classification_loss(logits ,target)
    et_loss = hpa_loss(w,x,et_map)
    return class_loss, l2_loss, et_loss

def cw_loss_fn(w,x, logits, target):
    l2_loss = ((.5*(torch.tanh(w) + 1)) - x ).flatten(-3).abs().norm(dim=-1)
    class_loss = classification_loss(logits ,target)
    return class_loss, l2_loss

def eta_attack(model, x, et_map, target, iterations, C=[1,1e-1,1e-2,1e-3,1e-4] ,D=[0.05,.1,.5], optim=torch.optim.Adam, lr=.01, verbose=False):
    '''
    This is a parallelized implementation of the eye-tracking-data aware CW-attack
    '''
    best_advs = torch.zeros_like(x)
    criterion = et_loss_fn
    best_loss = (torch.ones(x.shape[0]) * np.inf).to(device)
    for c in C:
        print(c)
        for d in D:
            print(d)
            w = torch.zeros_like(x).requires_grad_(True)            
            w.requires_grad_(True)
            optimizer = optim([w],lr=lr)
            for i in range(iterations):
                optimizer.zero_grad()
                adv = .5 * (torch.tanh(w)+1) #shape (224,224,3) --> (bs,224,224,3)
                logits = model(adv) #(10) --> (bs,10)
                pred = logits.argmax(-1) # (1) --> (bs,10)
                success = (pred == targets).bool()
                if verbose==True and i%200==0: print('iteration:', i)
                class_loss, l2_loss, hpa_loss = criterion(w,x,logits,et_map,target) #(1), (bs,1)
                raw_loss = l2_loss + hpa_loss
                loss = c * class_loss + l2_loss + d * hpa_loss # (1)-->(bs,1)
                loss = loss.sum()
                if i%50==0: 
                    print((best_advs.flatten(1) - img_batch.flatten(1)).norm(dim=-1).sum()/100)
                best_advs = adv.where((torch.logical_and(raw_loss<best_loss, success)).view(len(adv),1,1,1), best_advs)
                best_loss = raw_loss.where(torch.logical_and(raw_loss<best_loss, success), best_loss)
                #best_advs = torch.where(loss<best_loss, adv, best_advs)
                if i%50==0: 
                    print(torch.norm(best_advs-img_batch).item()/len(adv))

                loss.backward()
                optimizer.step()
                #stats = stats.append(pd.Series({'i':i, 'loss':loss.item()}), ignore_index=True)
    return best_advs#, stats

'''
def cw_attack(model, x, target, iterations, C=[1e-4,1e-3,1e-2,1e-1,1,10,100], optim=torch.optim.Adam, lr=.01, verbose=False):

    best_advs = torch.zeros_like(x)
    criterion = cw_loss_fn
    best_loss = (torch.ones(x.shape[0]) * np.inf).to(device)
    for c in C:
        w = torch.zeros_like(x).requires_grad_(True)            
        w.requires_grad_(True)
        optimizer = optim([w],lr=lr)
        for i in range(iterations):
            optimizer.zero_grad()
            adv = .5 * (torch.tanh(w)+1) #shape (224,224,3) --> (bs,224,224,3)
            logits = model(adv) #(10) --> (bs,10)
            pred = logits.argmax(-1) # (1) --> (bs,10)
            success = (pred == targets).bool()
            if verbose==True and i%200==0: print('iteration:', i)
            class_loss, l2_loss = criterion(w,x,logits,target,verbose=verbose) #(1), (bs,1)
            l2_loss = l2_loss
            loss = class_loss + c * l2_loss
            loss = loss.sum()
            if i%50==0: 
                print(l2_loss.sum().item(), loss.item())
            best_advs = adv.where((torch.logical_and(l2_loss<best_loss, success)).view(len(adv),1,1,1), best_advs)
            best_loss = l2_loss.where(torch.logical_and(l2_loss<best_loss, success), best_loss)
            #best_advs = torch.where(loss<best_loss, adv, best_advs)
            if i%50==0: 
                print(torch.norm(best_advs-img_batch).item()/len(adv))
            loss.backward()
            optimizer.step()
    return best_advs#, stats



def cw_attack(model, x, et_map, target, iterations, C=[1,1e-1,1e-2,1e-3,1e-4] ,D=[0], optim=torch.optim.Adam, lr=.05, verbose=False):

    best_advs = torch.zeros_like(x)
    criterion = et_loss_fn
    best_loss = (torch.ones(x.shape[0]) * np.inf).to(device)
    for c in C:
        print(c,'************')
        for d in D:
            print(d)
            w = torch.zeros_like(x).requires_grad_(True)            
            w.requires_grad_(True)
            optimizer = optim([w],lr=lr)
            for i in range(iterations):
                optimizer.zero_grad()
                adv = .5 * (torch.tanh(w)+1) #shape (224,224,3) --> (bs,224,224,3)
                logits = model(adv) #(10) --> (bs,10)
                pred = logits.argmax(-1) # (1) --> (bs,10)
                success = (pred == targets).bool()
                if verbose==True and i%200==0: print('iteration:', i)
                class_loss, l2_loss, hpa_loss = criterion(w,x,logits,et_map,target,verbose=verbose) #(1), (bs,1)
                raw_loss = l2_loss
                loss = c * class_loss + l2_loss
                loss = loss.sum()

                best_advs = adv.where((torch.logical_and(raw_loss<best_loss, success)).view(len(adv),1,1,1), best_advs)
                best_loss = raw_loss.where(torch.logical_and(raw_loss<best_loss, success), best_loss)
                #best_advs = torch.where(loss<best_loss, adv, best_advs)
                if i%50==0: 
                    print(torch.norm(best_advs-img_batch).item()/len(adv))
                
                loss.backward()
                optimizer.step()
                #stats = stats.append(pd.Series({'i':i, 'loss':loss.item()}), ignore_index=True)
    return best_advs#, stats
'''




def eta_attack_w_binary_search(model, x, et_map, target, iterations, D=[0.05,.1,.5,1,2], initial_const=1, search_steps=20,optim=torch.optim.Adam, lr=.01, verbose=False):
    best_advs = torch.zeros_like(x)
    criterion = et_loss_fn
    best_loss = (torch.ones(x.shape[0]) * np.inf).to(device)
    
    
    for d in D:
    
        #binary search for const
        c = (initial_const * torch.ones(x.shape[0])).to(device)
        upper = (torch.ones(x.shape[0]) * np.inf).to(device)
        lower = torch.zeros(x.shape[0]).to(device)

        for i in range(search_steps):
            w = torch.zeros_like(x).requires_grad_(True)
            w.requires_grad_(True)
            optimizer = optim([w],lr=lr)
            success_search_step = torch.zeros(x.shape[0]).bool().to(device)
            for i in range(iterations):
                optimizer.zero_grad()
                adv = .5 * (torch.tanh(w)+1) #shape (224,224,3) --> (bs,224,224,3)
                logits = model(adv) #(10) --> (bs,10)
                pred = logits.argmax(-1) # (1) --> (bs,10)
                success_iter = (pred == targets).bool()
                success_search_step = torch.logical_or(success_iter, success_search_step)

                if verbose==True and i%200==0: print('iteration:', i)
                class_loss, l2_loss, hpa_loss = criterion(w,x,logits,et_map,target) #(1), (bs,1)
                raw_loss = l2_loss + hpa_loss
                loss = c.to(device) * class_loss + l2_loss + d * hpa_loss # (1)-->(bs,1)
                loss = loss.sum()
                #if i%50==0: 
                #if True:
                    #print('l2',l2_loss[:5])
                    #print('best',best_loss[:5])
                   # 
                   # print('success',success_iter[:5])
                   # print('update', torch.logical_and(l2_loss<best_loss, success_iter)[:5])
                best_advs = adv.where((torch.logical_and(raw_loss<best_loss, success_iter)).view(len(adv),1,1,1), best_advs)
                best_loss = raw_loss.where(torch.logical_and(raw_loss<best_loss, success_iter), best_loss)
                #best_advs = torch.where(loss<best_loss, adv, best_advs)
                if i%50==0: 
                    print((best_advs.flatten(1) - img_batch.flatten(1)).norm(dim=-1).sum()/100)
                loss.backward()
                optimizer.step()
            #print((success_search_step == True)[:5])
            #print(c[:5])
            #print(((upper+lower)/2)[:5])
            old_lower = lower
            #print(success_search_step[:5])
            lower = lower.where(success_search_step, c)
            c = ((old_lower + c)/2).where(success_search_step, c*10)
            #print(c[:5])
            #c = ((c+lower)/2).where(success_search_step == True, c * 10)
            #print(c[:5])
            #with open('./adversarial_examples/et-aware-own-parallelize-implementation_w_binary_search.pkl', 'wb') as handle:
            #    pickle.dump(best_advs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return best_advs#, stats


def cw_attack(model, x, target, iterations, initial_const=1, search_steps=20,optim=torch.optim.Adam, lr=.01, verbose=False):
    best_advs = torch.zeros_like(x)
    criterion = cw_loss_fn
    best_loss = (torch.ones(x.shape[0]) * np.inf).to(device)

    #binary search for const
    c = (initial_const * torch.ones(x.shape[0])).to(device)
    upper = (torch.ones(x.shape[0]) * np.inf).to(device)
    lower = torch.zeros(x.shape[0]).to(device)
    for i in range(search_steps):
        w = torch.zeros_like(x).requires_grad_(True)
        w.requires_grad_(True)
        optimizer = optim([w],lr=lr)
        success_search_step = torch.zeros(x.shape[0]).bool().detach()
        #for early stopping
        #search_step_losses = np.array([])
        for i in range(iterations):
            optimizer.zero_grad()
            adv = .5 * (torch.tanh(w)+1) #shape (224,224,3) --> (bs,224,224,3)
            logits = model(adv) #(10) --> (bs,10)
            pred = logits.argmax(-1) # (1) --> (bs,10)
            success_iter = (pred == targets).bool().detach()
            success_search_step = torch.logical_or(success_iter.cpu(), success_search_step).detach()
            #if verbose==True and i%200==0: print('iteration:', i)
            class_loss, l2_loss = criterion(w,x,logits,target) #(1), (bs,1)
            loss = c.to(device) * class_loss + l2_loss
            loss = loss.sum()

            best_advs = adv.where((torch.logical_and(l2_loss<best_loss, success_iter)).view(len(adv),1,1,1), best_advs)
            best_loss = l2_loss.where(torch.logical_and(l2_loss<best_loss, success_iter), best_loss)
            #early abort mechanics - should work someting like this, if the loss did not decrease for any images in the 
            #batch within the last x iterations, break this loop
            #current_total_loss = best_loss.abs().sum()
            #print(current_total_loss.dtype)

            #if i>100 and (current_total_loss>=search_step_losses[-50:]).all() == True:
            #    print('early stopped')
                
            #    break
            #search_step_losses = np.append(search_step_losses, current_total_loss)
            #print(search_step_losses)
            
            #best_advs = torch.where(loss<best_loss, adv, best_advs)
            if i%50==0: 
                print((best_advs.flatten(1) - img_batch.flatten(1)).norm(dim=-1).sum()/100)
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()
        
        
        
        #print((success_search_step == True)[:5])
        #print(c[:5])
        #print(((upper+lower)/2)[:5])
        old_lower = lower.detach()
        #print(success_search_step[:5])
        lower = lower.cpu().where(success_search_step.cpu(), c.cpu()).detach()
        c = ((old_lower.cpu() + c.cpu())/2).cpu().where(success_search_step.cpu(), (c*10).cpu()).detach()
        #print(c[:5])
        #c = ((c+lower)/2).where(success_search_step == True, c * 10)
        torch.cuda.empty_cache()
        print(torch.norm(best_advs.flatten()-img_batch.flatten().detach()).item()/len(adv.detach()))
    return best_advs#, stats



In [6]:
print(str(datetime.now()))
advs = cw_attack(model, img_batch, targets, 10000, search_steps=20)
with open('./adversarial_examples/cw-own-parallelize-implementation_w_binary_search.pkl', 'wb') as handle:
    pickle.dump(advs, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(str(datetime.now()))

2021-12-22 13:52:50.351954
tensor(193.6514, device='cuda:0', grad_fn=<DivBackward0>)
tensor(90.8313, device='cuda:0', grad_fn=<DivBackward0>)
tensor(68.9348, device='cuda:0', grad_fn=<DivBackward0>)
tensor(51.8384, device='cuda:0', grad_fn=<DivBackward0>)
tensor(38.5868, device='cuda:0', grad_fn=<DivBackward0>)
tensor(28.6487, device='cuda:0', grad_fn=<DivBackward0>)
tensor(21.3823, device='cuda:0', grad_fn=<DivBackward0>)
tensor(15.8953, device='cuda:0', grad_fn=<DivBackward0>)
tensor(11.9879, device='cuda:0', grad_fn=<DivBackward0>)
tensor(9.2930, device='cuda:0', grad_fn=<DivBackward0>)
tensor(7.2052, device='cuda:0', grad_fn=<DivBackward0>)
tensor(5.8372, device='cuda:0', grad_fn=<DivBackward0>)
tensor(4.7535, device='cuda:0', grad_fn=<DivBackward0>)
tensor(4.0435, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.4443, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.0252, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.6911, device='cuda:0', grad_fn=<DivBackward0>)
tensor(2.41

tensor(0.8116, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8115, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8111, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8104, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8101, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8095, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8089, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8088, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8088, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8085, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8085, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8084, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8080, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8079, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8079, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8073, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8061, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.8060, device='cuda:0', grad_fn=<DivBack

tensor(0.7901, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7901, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7901, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7901, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7899, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7896, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7895, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7895, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7895, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7895, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7895, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7895, device='cuda:0', grad_fn=<DivBack

tensor(0.7255, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7246, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7237, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7235, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7229, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7224, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7223, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7222, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7221, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7219, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7219, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7218, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7213, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7206, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7202, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7201, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7199, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7197, device='cuda:0', grad_fn=<DivBack

tensor(0.7022, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7020, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7019, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7019, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7019, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7019, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7017, device='cuda:0', grad_fn=<DivBackward0>)
0.07420891761779785
tensor(0.7017, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7017, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7017, device='cuda:

...

tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
0.03491777181625366
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:

tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBack

tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBack

tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBack

tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.3183, device='cuda:0', grad_fn=<DivBack

In [ ]:
advs = eta_attack(
    model, 
    img_batch, 
    etm_batch, 
    targets, 
    10000,
    D=[0.05,.1,.5,1,2],
    optim=torch.optim.Adam, 
    lr=.01, 
    verbose=False
)

with open('./adversarial_examples/et-aware-own-parallelize-implementation.pkl', 'wb') as handle:
    pickle.dump(advs, handle, protocol=pickle.HIGHEST_PROTOCOL)

1
0.05
tensor(201.6759, device='cuda:0')
20.33070068359375
tensor(91.3169, device='cuda:0', grad_fn=<DivBackward0>)
9.430963134765625
tensor(69.2908, device='cuda:0', grad_fn=<DivBackward0>)
7.307010498046875
tensor(51.9706, device='cuda:0', grad_fn=<DivBackward0>)
5.625177612304688
tensor(38.6295, device='cuda:0', grad_fn=<DivBackward0>)
4.3055517578125
tensor(28.5271, device='cuda:0', grad_fn=<DivBackward0>)
3.2971343994140625
tensor(21.1174, device='cuda:0', grad_fn=<DivBackward0>)
2.5353578186035155
tensor(15.6362, device='cuda:0', grad_fn=<DivBackward0>)
1.9551956176757812
tensor(11.8002, device='cuda:0', grad_fn=<DivBackward0>)
1.5390013122558595
tensor(9.0502, device='cuda:0', grad_fn=<DivBackward0>)
1.2214064025878906
tensor(7.0593, device='cuda:0', grad_fn=<DivBackward0>)
0.9859077453613281
tensor(5.7240, device='cuda:0', grad_fn=<DivBackward0>)
0.8209598541259766
tensor(4.6460, device='cuda:0', grad_fn=<DivBackward0>)
0.67093505859375
tensor(3.9291, device='cuda:0', grad_fn=<

tensor(0.8350, device='cuda:0', grad_fn=<DivBackward0>)
0.08929121971130372
tensor(0.8335, device='cuda:0', grad_fn=<DivBackward0>)
0.08911582946777344
tensor(0.8315, device='cuda:0', grad_fn=<DivBackward0>)
0.08884983062744141
tensor(0.8311, device='cuda:0', grad_fn=<DivBackward0>)
0.0887965965270996
tensor(0.8300, device='cuda:0', grad_fn=<DivBackward0>)
0.08872156143188477
tensor(0.8284, device='cuda:0', grad_fn=<DivBackward0>)
0.0885261344909668
tensor(0.8281, device='cuda:0', grad_fn=<DivBackward0>)
0.08850863456726074
tensor(0.8275, device='cuda:0', grad_fn=<DivBackward0>)
0.08846259117126465
tensor(0.8266, device='cuda:0', grad_fn=<DivBackward0>)
0.08834604263305663
tensor(0.8265, device='cuda:0', grad_fn=<DivBackward0>)
0.08834324836730957
tensor(0.8262, device='cuda:0', grad_fn=<DivBackward0>)
0.08832188606262208
tensor(0.8258, device='cuda:0', grad_fn=<DivBackward0>)
0.08830195426940918
tensor(0.8252, device='cuda:0', grad_fn=<DivBackward0>)
0.08825859069824218
tensor(0.8254,

tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8009, device='cuda:0', grad_fn=<DivBackward0>)
0.08575340270996094
tensor(0.8011, device='cuda:0', grad_fn=<DivBackward0>)
0.0857720947265625
tensor(0.8011

tensor(0.8013, device='cuda:0', grad_fn=<DivBackward0>)
0.08581684112548828
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08567914962768555
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568043708801269
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568043708801269
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08567862510681153
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568037033081055
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568037033081055
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568018913269043
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568018913269043
tensor(0.8002, device='cuda:0', grad_fn=<DivBackward0>)
0.08568018913269043
tensor(0.7995, device='cuda:0', grad_fn=<DivBackward0>)
0.08557281494140626
tensor(0.7996, device='cuda:0', grad_fn=<DivBackward0>)
0.0855785083770752
tensor(0.7994, device='cuda:0', grad_fn=<DivBackward0>)
0.08556836128234863
tensor(0.7994

tensor(0.8068, device='cuda:0', grad_fn=<DivBackward0>)
0.08652219772338868
tensor(0.8060, device='cuda:0', grad_fn=<DivBackward0>)
0.08642574310302735
tensor(0.8060, device='cuda:0', grad_fn=<DivBackward0>)
0.08642574310302735
tensor(0.8060, device='cuda:0', grad_fn=<DivBackward0>)
0.08642574310302735
tensor(0.8066, device='cuda:0', grad_fn=<DivBackward0>)
0.08649950981140136
tensor(0.8066, device='cuda:0', grad_fn=<DivBackward0>)
0.08649950981140136
tensor(0.8091, device='cuda:0', grad_fn=<DivBackward0>)
0.08693431854248047
tensor(0.8118, device='cuda:0', grad_fn=<DivBackward0>)
0.08727210998535156
tensor(0.8123, device='cuda:0', grad_fn=<DivBackward0>)
0.08724825859069824
tensor(0.8121, device='cuda:0', grad_fn=<DivBackward0>)
0.08721195220947266
tensor(0.8131, device='cuda:0', grad_fn=<DivBackward0>)
0.08730542182922363
tensor(0.8131, device='cuda:0', grad_fn=<DivBackward0>)
0.08725890159606933
tensor(0.8143, device='cuda:0', grad_fn=<DivBackward0>)
0.08749435424804687
tensor(0.814

...

tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.4046, device='cuda:0', grad_fn=<DivBackward0>)
0.04373063087463379
tensor(0.404

tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374575614929199
tensor(0.404

tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.404

tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.404

tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.4049, device='cuda:0', grad_fn=<DivBackward0>)
0.04374640464782715
tensor(0.404

In [ ]:
plt.imshow(img_batch[6].detach().cpu().permute(1,2,0))

In [ ]:
advs = eta_attack_w_binary_search(
    model, 
    img_batch, 
    etm_batch, 
    targets, 
    10000,
    D=[0.05,.1,.5,1,2],
    optim=torch.optim.Adam, 
    lr=.01, 
    verbose=False
)

with open('./adversarial_examples/et-aware-own-parallelize-implementation_w_binary_search.pkl', 'wb') as handle:
    pickle.dump(advs, handle, protocol=pickle.HIGHEST_PROTOCOL)